In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [16]:
df = pd.read_csv('cardekho_dataset.csv')

In [17]:
df.drop(['SNo'], axis=1, inplace=True)

In [18]:
df.isna().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [19]:
df.drop('car_name',axis=1 ,inplace=True)
df.drop('brand',axis=1 ,inplace=True)

In [20]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [21]:
num_features = [feature for feature in df.columns if df[feature].dtypes != 'O']
print('Number of numerical variables: ', len(num_features))
cat_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
print('Number of categorical variables: ', len(cat_features))
discrete_feature=[feature for feature in num_features if len(df[feature].unique())<25]
print("Discrete Variables Count: {}".format(len(discrete_feature)))
continuous_feature=[feature for feature in num_features if feature not in discrete_feature+['price']]
print("Continuous feature Count {}".format(len(continuous_feature)))

Number of numerical variables:  7
Number of categorical variables:  4
Discrete Variables Count: 2
Continuous feature Count 5


In [22]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [23]:
from sklearn.model_selection import train_test_split
X = df.drop('selling_price', axis=1)
y = df['selling_price']

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])


In [27]:
num_features = X.select_dtypes(exclude="object").columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, onehot_columns),
        ("StandardScaler", numeric_transformer, num_features)
    ],remainder='passthrough'
)

In [29]:
X = preprocessor.fit_transform(X)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((12328, 14), (3083, 14))

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [35]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mse, mae, r2,rmse

In [36]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNN": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor()
}

for i in range(len(models)):
    model = models[list(models.keys())[i]]
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    model_train_mae,model_train_mse,model_train_r2,model_train_rmse = evaluate_model(y_train, y_train_pred)
    model_test_mae,model_test_mse,model_test_r2,model_test_rmse = evaluate_model(y_test, y_test_pred)
    
    print(f"{list(models.keys())[i]} Model Evaluation")
    
    print(f"Training Mean Absolute Error: {model_train_mae}")
    print(f"Training Mean Squared Error: {model_train_mse}")
    print(f"Training R2 Score: {model_train_r2}")
    print(f"Training Root Mean Squared Error: {model_train_rmse}")
    
    print(f"Testing Mean Absolute Error: {model_test_mae}")
    print(f"Testing Mean Squared Error: {model_test_mse}")
    print(f"Testing R2 Score: {model_test_r2}")
    print(f"Testing Root Mean Squared Error: {model_test_rmse}")
    
    print("\n")

Linear Regression Model Evaluation
Training Mean Absolute Error: 306756099359.7596
Training Mean Squared Error: 268101.6070829937
Training R2 Score: 0.6217719576765959
Training Root Mean Squared Error: 553855.6665411663
Testing Mean Absolute Error: 252550062888.5656
Testing Mean Squared Error: 279618.57941584283
Testing R2 Score: 0.6645109298852004
Testing Root Mean Squared Error: 502543.5930230985


Lasso Model Evaluation
Training Mean Absolute Error: 306756104248.3742
Training Mean Squared Error: 268099.2226498115
Training R2 Score: 0.6217719516489697
Training Root Mean Squared Error: 553855.6709544231
Testing Mean Absolute Error: 252549134806.78134
Testing Mean Squared Error: 279614.7461034126
Testing R2 Score: 0.6645121627547996
Testing Root Mean Squared Error: 502542.66963789385


Ridge Model Evaluation
Training Mean Absolute Error: 306756818740.9266
Training Mean Squared Error: 268059.8014688311
Training R2 Score: 0.6217710706848424
Training Root Mean Squared Error: 553856.315970

In [37]:
knn_params = {"n_neighbors": [2,3,10,20,40,50]}
rf_params = {
    "max_depth": [None, 8, 5, 10,15],
    "min_samples_split": [2, 8, 15,20],
    "max_features": [5,7,"auto",8],
    "n_estimators": [100, 200, 500,1000]
}

In [38]:
randomcv_models = {
    "KNN": KNeighborsRegressor(),
    "Random Forest": RandomForestRegressor()
}

In [41]:
from sklearn.model_selection import RandomizedSearchCV

model_params = {}
for name, model in randomcv_models.items():
    if name == "KNN":
        params = knn_params
    elif name == "Random Forest":
        params = rf_params
    random = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter=100, cv=3, n_jobs=-1, verbose=2)
    random.fit(X_train, y_train)
    model_params[name] = random.best_params_

for model_name in model_params:
    print(f"{model_name} Best Parameters: {model_params[model_name]}") 

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
KNN Best Parameters: {'n_neighbors': 10}
Random Forest Best Parameters: {'n_estimators': 500, 'min_samples_split': 2, 'max_features': 8, 'max_depth': 15}
